In [106]:
import pandas as pd
import seaborn as sns
from docx import Document 

In [107]:
# data = pd.read_excel("../Base_filtrada.xlsx")
# data = pd.read_excel("../Bases/Base_filtrada_idade.xlsx")
# data = pd.read_excel("../Bases/Base_plano_Cartesiano.xlsx")
data = pd.read_excel("../Bases/Base_tratada.xlsx")

### Separação por natureza dos dados

filtrada

In [108]:
# colunas_categoricas = ['V0001', 'A02201', 'C004', 'C006', 'C009', 'L01701', 'L018', 'M01601', 'N010','P034',
#                        'P050', 'P052', 'Q092', 'Q11006', 'Q11604', 'Q074']

In [109]:
# colunas_numericas = ['V0022', 'A02305', 'A02306', 'C008', 'P00104', 'P00404', 'P035', 'P03701', 'P03702', 
#                      'P053', 'P05901', 'P05902', 'P05903', 'P05904']  

plano cartesiano

In [110]:
# colunas_categoricas = ['V0001', 'C004', 'C006', 'C009', 'L01701', 'L018', 'M01601', 'N010','P034|P035', 'P03701|P03702',
#                        'P050|P052', 'P05901|P05902|P05903|P05904', 'Q092', 'Q11006', 'Q11604', 'Q074']

In [111]:
# colunas_numericas = ['V0022', 'A02201|A02305|A02306', 'C008', 'P00104', 'P00404', 'P053']  

tratada

In [112]:
colunas_categoricas = ['V0001','V0022', 'A02201|A02305|A02306', 'C008','C006', 'C009', 'N010', 'P00104', 'P00404', 'P03701|P03702','IMC',
                       'P050|P052', 'P053', 'P05901|P05902|P05903|P05904', 'Q092', 'Q11604', 'Q074']

In [113]:
colunas_numericas = ['P034|P035']  

garantir que as colunas numericas sao numericas

In [114]:
for coluna in colunas_numericas:
    data[coluna] = pd.to_numeric(data[coluna], errors='coerce')

## Correlação

## Numero de faltantes

In [115]:
for col in data.columns:
    nulos = data[col].isnull().sum()
    print(f'numero de valores nulos na coluna {col} é: {nulos}')

numero de valores nulos na coluna V0001 é: 0
numero de valores nulos na coluna V0022 é: 0
numero de valores nulos na coluna A02201|A02305|A02306 é: 0
numero de valores nulos na coluna C006 é: 0
numero de valores nulos na coluna C008 é: 0
numero de valores nulos na coluna C009 é: 0
numero de valores nulos na coluna N010 é: 0
numero de valores nulos na coluna P00104 é: 0
numero de valores nulos na coluna P00404 é: 0
numero de valores nulos na coluna P034|P035 é: 0
numero de valores nulos na coluna P03701|P03702 é: 0
numero de valores nulos na coluna P050|P052 é: 0
numero de valores nulos na coluna P053 é: 0
numero de valores nulos na coluna P05901|P05902|P05903|P05904 é: 0
numero de valores nulos na coluna Q074 é: 0
numero de valores nulos na coluna Q092 é: 0
numero de valores nulos na coluna Q11604 é: 0
numero de valores nulos na coluna IMC é: 0


In [116]:
for col in data.columns:
    nulos = data[col].isna().sum()
    print(f'numero de valores nulos na coluna {col} é: {nulos}')

numero de valores nulos na coluna V0001 é: 0
numero de valores nulos na coluna V0022 é: 0
numero de valores nulos na coluna A02201|A02305|A02306 é: 0
numero de valores nulos na coluna C006 é: 0
numero de valores nulos na coluna C008 é: 0
numero de valores nulos na coluna C009 é: 0
numero de valores nulos na coluna N010 é: 0
numero de valores nulos na coluna P00104 é: 0
numero de valores nulos na coluna P00404 é: 0
numero de valores nulos na coluna P034|P035 é: 0
numero de valores nulos na coluna P03701|P03702 é: 0
numero de valores nulos na coluna P050|P052 é: 0
numero de valores nulos na coluna P053 é: 0
numero de valores nulos na coluna P05901|P05902|P05903|P05904 é: 0
numero de valores nulos na coluna Q074 é: 0
numero de valores nulos na coluna Q092 é: 0
numero de valores nulos na coluna Q11604 é: 0
numero de valores nulos na coluna IMC é: 0


## Tabela numerica

In [117]:
estatisticas_numericas = {
    'Coluna': colunas_numericas,
    'Mínimo': [round(data[col].min(), 2) for col in colunas_numericas],
    'Máximo': [round(data[col].max(), 2) for col in colunas_numericas],
    'Média': [round(data[col].mean(), 2) for col in colunas_numericas],
    'Desvio Padrão': [round(data[col].std(), 2) for col in colunas_numericas],
    'Dados Faltantes': [data[col].isnull().sum() for col in colunas_numericas],
    '% Faltantes': [round((data[col].isnull().sum() / len(data)) * 100, 2) for col in colunas_numericas]
}
df_estatisticas_numericas = pd.DataFrame(estatisticas_numericas)

df_estatisticas_numericas = pd.DataFrame(estatisticas_numericas)

## Tabela Categorica

In [ ]:
from math import log2

def calcular_entropia_condicional(df, coluna, classe):
    grupos = df.groupby(coluna)
    total = len(df)
    entropia_total = 0.0
    for _, grupo in grupos:
        proporcao_grupo = len(grupo) / total
        dist_classe = grupo[classe].value_counts(normalize=True)
        entropia = -sum(p * log2(p) for p in dist_classe if p > 0)
        entropia_total += proporcao_grupo * entropia
    return round(entropia_total, 4)

estatisticas_categoricas_detalhadas = {
    'Coluna': [],
    'Categoria': [],
    'Contagem': [],
    '% da Coluna': [],
    'Instâncias por Classe': [],
    'Entropia com Classe': []
}

for coluna in colunas_categoricas:
    contagem = data[coluna].value_counts(dropna=False).reset_index()
    contagem.columns = ['Categoria', 'Contagem']

    try:
        contagem['Categoria_Num'] = pd.to_numeric(contagem['Categoria'], errors='coerce')
        contagem = contagem.sort_values(by='Categoria_Num').drop('Categoria_Num', axis=1)
    except:
        contagem = contagem.sort_values(by='Categoria')
    
    total_linhas = len(data)
    
    # Cálculo de correlação e entropia com a classe Q074
    matriz_confusao = pd.crosstab(data[coluna], data['Q074'])
    entropia = calcular_entropia_condicional(data[[coluna]].assign(Q074=data['Q074']), coluna, 'Q074')

    for _, row in contagem.iterrows():
        categoria = row['Categoria']
        dados_categoria = data[data[coluna] == categoria]
        dist_classe = dados_categoria['Q074'].value_counts().to_dict()
        dist_classe_str = ', '.join([f'{k}: {v}' for k, v in dist_classe.items()])
        
        estatisticas_categoricas_detalhadas['Coluna'].append(coluna)
        estatisticas_categoricas_detalhadas['Categoria'].append(str(categoria))
        estatisticas_categoricas_detalhadas['Contagem'].append(row['Contagem'])
        estatisticas_categoricas_detalhadas['% da Coluna'].append(round((row['Contagem'] / total_linhas) * 100, 2))
        estatisticas_categoricas_detalhadas['Instâncias por Classe'].append(dist_classe_str)
        estatisticas_categoricas_detalhadas['Entropia com Classe'].append(entropia)

# Criar DataFrame final
df_categoricas_ordenado = pd.DataFrame(estatisticas_categoricas_detalhadas)



AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
# def adicionar_teste_chi2_doc(doc, data, colunas_categoricas, classe='Q074'):
#     from itertools import combinations
#     from scipy.stats import chi2_contingency

#     doc.add_heading('Teste Qui-Quadrado: Comparações Dois a Dois', level=1)

#     for coluna in colunas_categoricas:
#         categorias_unicas = data[coluna].dropna().unique()
#         combinacoes = list(combinations(categorias_unicas, 2))
        
#         if not combinacoes:
#             continue  # pula colunas sem pelo menos duas categorias

#         doc.add_heading(f'Coluna: {coluna}', level=2)
#         tabela = doc.add_table(rows=1, cols=4)
#         hdr_cells = tabela.rows[0].cells
#         hdr_cells[0].text = 'Categoria 1'
#         hdr_cells[1].text = 'Categoria 2'
#         hdr_cells[2].text = 'Valor-p'
#         hdr_cells[3].text = 'Hipótese Nula'

#         for cat1, cat2 in combinacoes:
#             subset = data[data[coluna].isin([cat1, cat2])]
#             contingencia = pd.crosstab(subset[coluna], subset[classe])

#             # Testar apenas se for pelo menos 2x2
#             if contingencia.shape[0] == 2 and contingencia.shape[1] >= 2:
#                 try:
#                     _, p_valor, _, _ = chi2_contingency(contingencia)
#                     p_valor_ajustado = round(p_valor, 4)
#                     resultado = 'Rejeitada' if p_valor < 0.05 else 'Aceita'
                    
#                     row_cells = tabela.add_row().cells
#                     row_cells[0].text = str(cat1)
#                     row_cells[1].text = str(cat2)
#                     row_cells[2].text = str(p_valor_ajustado)
#                     row_cells[3].text = resultado
#                 except:
#                     continue  # em caso de erro com os dados, pula o par

## Analise

In [ ]:
print("Análise Descritiva do Conjunto de Dados Numéricos")
print(df_estatisticas_numericas)

print("Análise Descritiva do Conjunto de Dados Categóricos")


Análise Descritiva do Conjunto de Dados Numéricos
      Coluna  Mínimo  Máximo  Média  Desvio Padrão  Dados Faltantes  \
0  P034|P035       0       7   1.66           2.12                0   

   % Faltantes  
0          0.0  
Análise Descritiva do Conjunto de Dados Categóricos


## Salvando em arquivo

In [ ]:
doc = Document()
doc.add_heading('Análise Descritiva do Conjunto de Dados', 0)

In [ ]:
doc.add_heading('Estatísticas Numéricas', level=1)
tabela_numericas = doc.add_table(rows=1, cols=len(df_estatisticas_numericas.columns))

In [ ]:
hdr_cells = tabela_numericas.rows[0].cells

for i, col in enumerate(df_estatisticas_numericas.columns):
    hdr_cells[i].text = str(col)


In [ ]:
for index, row in df_estatisticas_numericas.iterrows():
    row_cells = tabela_numericas.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

In [ ]:
doc.add_heading('Análise Categórica', level=1)

for coluna in colunas_categoricas:
    doc.add_heading(f'Coluna: {coluna}', level=2)
    dados_coluna = df_categoricas_ordenado[df_categoricas_ordenado['Coluna'] == coluna]
    
    tabela = doc.add_table(rows=1, cols=6)
    hdr_cells = tabela.rows[0].cells
    hdr_cells[0].text = 'Categoria'
    hdr_cells[1].text = 'Contagem'
    hdr_cells[2].text = '% da Coluna'
    hdr_cells[3].text = 'Instâncias por Classe'
    hdr_cells[4].text = 'Entropia com Classe'

    for _, row in dados_coluna.iterrows():
        row_cells = tabela.add_row().cells
        row_cells[0].text = str(row['Categoria'])
        row_cells[1].text = str(row['Contagem'])
        row_cells[2].text = f"{row['% da Coluna']}%"
        row_cells[3].text = row['Instâncias por Classe']
        row_cells[4].text = str(row['Entropia com Classe'])

In [ ]:
# adicionar_teste_chi2_doc(doc, data, colunas_categoricas, classe='Q074')

In [ ]:
# doc.save('../Relatorios/tabela_estatisticas.docx')
# doc.save('../Relatorios/tabela_estatisticas_filtrada.docx')
# doc.save('../Relatorios/tabela_estatisticas_plano_cartesiano.docx')
doc.save('../Relatorios/tabela_base_tratada.docx')